In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mltiverse-join-comp/MLtiVerse/sample_submission.csv
/kaggle/input/mltiverse-join-comp/MLtiVerse/train.csv
/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv


importing the data needed for hackathon

In [2]:
train_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/train.csv')
test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv')
sub_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/sample_submission.csv')

/tmp/ipykernel_20/1580012756.py:2: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv')


viewing the id column

In [3]:
train_data['Photo'].head()

0    https://cdn.sofifa.com/players/195/427/17_60.png
1    https://cdn.sofifa.com/players/203/539/17_60.png
2    https://cdn.sofifa.com/players/232/898/20_60.png
3    https://cdn.sofifa.com/players/215/502/17_60.png
4    https://cdn.sofifa.com/players/222/263/18_60.png
Name: Photo, dtype: object

going to start with a very basic xgboost submission selecting what i think are some important columns

importing xgboost model

In [4]:
from xgboost import XGBRegressor
model=XGBRegressor()

checking which columns are available for modelling

In [5]:
train_data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'year', 'Release Clause',
       'DefensiveAwareness'],
 

In [6]:
train_data['Release Clause'].isna().sum()

25095

In [7]:
train_data['Potential'].head()

0    68
1    58
2    76
3    86
4    75
Name: Potential, dtype: int64

In [8]:
train_data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'year', 'Release Clause',
       'DefensiveAwareness'],
 

manually selecting some important features in the dataset for modelling

In [9]:
train_1=train_data[['Potential','Overall','Wage','International Reputation','Age','year']]

In [10]:
train_1['Wage'].head()

0     €2K
1    €500
2     €7K
3    €10K
4     €2K
Name: Wage, dtype: object

making some changes to the wage column

In [11]:
# Remove euro symbol and 'K', and convert to float
train_1['Wage'] = train_1['Wage'].str.replace('€', '')  # Remove euro symbol
train_1['Wage'] = train_1['Wage'].str.replace('K', '').astype(float) * 1000  # Remove 'K' and multiply by 1000

# Convert the column to float data type
train_1['Wage'] = train_1['Wage'].astype(float)

/tmp/ipykernel_20/2962417934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].str.replace('€', '')  # Remove euro symbol
/tmp/ipykernel_20/2962417934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].str.replace('K', '').astype(float) * 1000  # Remove 'K' and multiply by 1000
/tmp/ipykernel_20/2962417934.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [12]:
train_1['Wage'].isna().sum()

0

checking which data types are object

In [13]:
object_columns = train_1.select_dtypes(include='object').columns
object_columns

Index([], dtype='object')